In [ ]:
from netCDF4 import Dataset
import numpy as np
import xarray as xr


ds = xr.open_dataset("/hades/skhan/repos/MLFF/data/shnitsel/fixed/A03_butene_0p50fs_dynamic.nc")
ds

<xarray.Dataset> Size: 134MB
Dimensions:    (state: 3, frame: 36579, statecomb: 3, atom: 12, direction: 3,
                trajid_: 156)
Coordinates:
  * state      (state) int64 24B 1 2 3
  * atom       (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction  (direction) <U1 12B 'x' 'y' 'z'
  * trajid_    (trajid_) int32 624B 2 3 6 11 13 16 ... 307 310 311 313 314 316
    atNames    (atom) <U1 48B ...
    trajid     (frame) int64 293kB ...
    time       (frame) float64 293kB ...
    from       (statecomb) int64 24B ...
    to         (statecomb) int64 24B ...
Dimensions without coordinates: frame, statecomb
Data variables: (12/13)
    energy     (state, frame) float64 878kB ...
    nacs       (statecomb, atom, direction, frame) float64 32MB ...
    atXYZ      (atom, direction, frame) float64 11MB ...
    sdiag      (frame) int64 293kB ...
    phases     (state, frame) float64 878kB ...
    e_kin      (frame) float64 293kB ...
    ...         ...
    dip_perm   (state, direction, frame) float64 3MB ...
    dip_trans  (statecomb, direction, frame) float64 3MB ...
    astate     (frame) int64 293kB ...
    completed  (frame, trajid_) int8 6MB ...
    nsteps     (frame, trajid_) int32 23MB ...
    max_ts     (frame, trajid_) int32 23MB ...
Attributes:
    delta_t:                           0.5
    _MultiIndex_levels_for_frame:      ['trajid', 'time']
    _MultiIndex_levels_for_statecomb:  ['from', 'to']
    _MultiIndex_levels_from_attrs:     1

In [14]:
print("VARS:", list(ds.variables.keys()))


VARS: ['energy', 'nacs', 'atXYZ', 'sdiag', 'phases', 'e_kin', 'forces', 'dip_perm', 'dip_trans', 'astate', 'completed', 'nsteps', 'max_ts', 'state', 'atom', 'direction', 'atNames', 'trajid_', 'trajid', 'time', 'from', 'to']


In [15]:

# list all available variables
print("Variables:", list(ds.variables.keys()))

# read the 'state' variable (list of states available)
states = ds.variables["state"][:]   # shape: (n_states,)
print("Electronic states:", states)

# print how many total states and maybe first few energy values
energies = ds.variables["energy"][:]  # shape: (n_states, n_frames)
print(f"Number of electronic states: {energies.shape[0]}")
print("First 5 energy values per state:")
for i, e in enumerate(energies[:,:5]):
    print(f"State {i}: {e}")

# read the active state per frame (which state was active at each time step)
active = ds.variables["astate"][:]
print("Active state indices:", active[:20])


Variables: ['energy', 'nacs', 'atXYZ', 'sdiag', 'phases', 'e_kin', 'forces', 'dip_perm', 'dip_trans', 'astate', 'completed', 'nsteps', 'max_ts', 'state', 'atom', 'direction', 'atNames', 'trajid_', 'trajid', 'time', 'from', 'to']
Electronic states: <xarray.IndexVariable 'state' (state: 3)> Size: 24B
array([1, 2, 3])
Number of electronic states: 3
First 5 energy values per state:
State 0: <xarray.Variable (frame: 5)> Size: 40B
[5 values with dtype=float64]
Attributes:
    units:    hartree
    unitdim:  Energy
State 1: <xarray.Variable (frame: 5)> Size: 40B
[5 values with dtype=float64]
Attributes:
    units:    hartree
    unitdim:  Energy
State 2: <xarray.Variable (frame: 5)> Size: 40B
[5 values with dtype=float64]
Attributes:
    units:    hartree
    unitdim:  Energy
Active state indices: <xarray.Variable (frame: 20)> Size: 160B
[20 values with dtype=int64]
Attributes:
    long_name:  active state (MCH)


In [ ]:
data = np.load("/hades/skhan/repos/MLFF/data/shnitsel/fixed/A03_butene_0p50fs_dynamic.npz")
print(data.files)
for k in data.files:
    print(k, data[k].shape)

['R', 'F', 'E', 'z', 'astate', 'energy_all', 'forces_all']
R (36579, 12, 3)
F (36579, 12, 3)
E (36579,)
z (12,)
astate (36579,)
energy_all (3, 36579)
forces_all (3, 12, 3, 36579)


In [17]:
print(data["R"].shape)  
print(data["E"].shape)

(36579, 12, 3)
(36579,)


In [ ]:
data = np.load("/hades/skhan/repos/MLFF/data/shnitsel/fixed/A03_butene_0p50fs_dynamic_filtered_state_3.npz")
print(data.files)
for k in data.files:
    print(k, data[k].shape)

['R', 'F', 'E', 'z']
R (19, 12, 3)
F (19, 12, 3)
E (19,)
z (19, 12)


### Generating data for Delta model

In [ ]:
filtering_data = np.load("/hades/skhan/repos/MLFF/data/shnitsel/fixed/A03_butene_0p50fs_dynamic.npz")

R          = filtering_data["R"]          # (n_frames, n_atoms, 3)
z          = filtering_data["z"]  # (n_atoms,)
energy_all = data["energy_all"] # (n_states, n_frames)
forces_all = data["forces_all"] # (n_states, n_atoms, 3, n_frames)

In [19]:
E0 = energy_all[0]  # (n_frames,)
E1 = energy_all[1]  # (n_frames,)
E2 = energy_all[2]  # (n_frames,)

deltaE_01 = E1 - E0  # (n_frames,)
deltaE_02 = E2 - E0  # (n_frames,)


In [21]:
deltaE_02

array([0.4904755, 0.4950446, 0.4969546, ..., 0.4370323, 0.4557639,
       0.4732503], shape=(36579,))

In [20]:
F0 = forces_all[0]  # (n_atoms, 3, n_frames)
F1 = forces_all[1]  # (n_atoms, 3, n_frames)

deltaF_01 = F1 - F0             # (n_atoms, 3, n_frames)
deltaF_01 = np.moveaxis(deltaF_01, -1, 0)  # → (n_frames, n_atoms, 3)
